In [1]:
import random
import datetime 
import pandas as pd
from faker import Faker

Creating class-objects for Houses and Applicants

In [94]:
class House:
    shortlist = pd.DataFrame(columns=['id','priority','bp_date','ap_date'])
    def __init__(self,id,rooms,status,bids,slist=shortlist,tenant=0):
        self.id = id
        self.rooms = rooms
        self.status = status
        self.bids = bids
        self.slist = slist
        self.tenant = tenant
    
    def __str__(self):
        return f"House ID:{self.id} has {self.rooms} rooms and {len(self.bids)} bid(s). It is currently {self.status}"

    def shortlister(self):
        shortlist = pd.DataFrame(columns=['id','priority','bp_date','ap_date'])
        for x in self.bids:
            if applicants[x].status == 'Live':
                shortlist.loc[x,'id'] = applicants[x].id
                shortlist.loc[x,'priority'] = applicants[x].priority
                shortlist.loc[x,'bp_date'] = applicants[x].bp_date
                shortlist.loc[x, 'ap_date'] = applicants[x].ap_date
            else:
                continue
        
        shortlist.sort_values(['priority','bp_date','ap_date'],axis=0,inplace=True,ignore_index=True)
        self.slist = shortlist
        top = self.slist.loc[0,'id']
        applicants[top].offers.append(self.id)
        print(top)
        return shortlist
    
    def contract(self):
        top = self.slist.loc[0,'id']
        if applicants[top].home == self.id:
            self.status = 'Occupied'
            self.tenant = top
            applicants[top].status = 'Housed'
            return f'House {self.id} housed Applicant {self.tenant}'
        else:
            pass



            
        
        

In [108]:
class Applicant:
    def __init__(self,id,priority,bp_date,ap_date,hsize,status,bids=[],offers=[],home=0):
        self.id = id
        self.priority = priority
        self.bp_date = bp_date
        self.ap_date = ap_date
        self.hsize = hsize
        self.status = status
        self.bids = bids
        self.offers = offers
        self.home = home

    def __str__(self):
        return f"Applicant {self.id} is in Priority Band {self.priority}. They have a need of {self.hsize} rooms and are currently {self.status}"
    
    def bidder(self,hlist):
        shortlist = []
        for x in hlist:
            if hlist[x].rooms == self.hsize:
                shortlist.append((x,hlist[x]))
        try:
            bids = random.sample(shortlist,k=3)
        except ValueError:
            bids = random.sample(shortlist,k=len(shortlist))
        for x in bids:
            houses[x[0]].bids.append(self.id)
        self.bids = bids 
        return bids
    
    def shortlist(self):
        if len(self.offers) > 1:
            self.home = random.choice(self.offers)
            return self.home
        else:
            self.home = self.offers[0]
            return self.home
        
    


Generate random applicants and houses

In [5]:
def date_generator(y,m,d):
    fake = Faker()
    start_date = datetime.datetime(y,m,d)
    return fake.date_between(start_date=start_date, end_date='now')

def house_generator(ind):
    id = ind
    rooms  = random.randint(0,5)
    house = House(id,rooms,'Live',[])
    return house

def applicant_generator(ind):
    id = ind
    priority = random.randint(1,4)
    bp_date = date_generator(2015,1,1)
    ap_date = date_generator(2021,9,19)
    hsize = random.randint(0,5)
    applicant = Applicant(id,priority,bp_date,ap_date,hsize,'Live')
    return applicant

In [110]:
try:
    del houses
except NameError:
    pass
houses = {}
for x in range(1,21):
    houses[x] = house_generator(x)

In [125]:
try:
    del applicants
except NameError:
    pass
applicants = {}
for x in range(1,201):
    applicants[x] = applicant_generator(x)

A typical biddinng cycle consists of a round of bidding, then landlords shortlist tenants and make an offer to the top applicant, the applicant picks which home to move to if they receive multiple offers. The landlord then picks the next most suitable tenant, and the cycle continues till all houses are occupied

In [111]:
homes = pd.DataFrame(columns=['id','rooms','status'])
for x in houses:
    homes.loc[x,'id'] = houses[x].id
    homes.loc[x,'rooms'] = houses[x].rooms
    homes.loc[x,'status'] = houses[x].status

In [112]:
for x in houses:
    homes.loc[x,'status'] = houses[x].status
homes

,id,rooms,status
1,1,4,Live
2,2,4,Live
3,3,1,Live
4,4,1,Live
5,5,4,Live
6,6,0,Live
7,7,1,Live
8,8,2,Live
9,9,3,Live
10,10,2,Live


In [113]:
for x in applicants:
    applicants[x].bidder(houses)

In [117]:
for x in houses:
    houses[x].shortlister()

162
150
144
114
150
123
144
69
199
69
123
150
86
199
4
162
86
132
144
123


In [75]:
for x in applicants:
    applicants[x].shortlist()

In [76]:
applicants[156].home

1

In [77]:
for x in houses:
    houses[x].contract()

In [79]:
houses[1].slist

,id,priority,bp_date,ap_date
0,51,1,2015-05-15,2022-02-12
1,161,1,2016-10-15,2021-10-14
2,135,1,2019-08-26,2022-06-25
3,125,1,2020-07-08,2021-10-30
4,184,1,2021-09-16,2022-04-15
5,191,2,2016-01-06,2021-11-13
6,155,2,2016-01-28,2021-12-31
7,70,2,2017-01-03,2022-02-27
8,20,2,2018-03-07,2021-12-14
9,71,2,2018-05-03,2022-02-20


In [32]:
cycle = 0
while 'Live' in homes['status'].unique():
    for x in applicants:
        applicants[x].bidder(houses)
    for x in houses:
        houses[x].shortlister()
    for x in houses:
        houses[x].offer()
    for x in applicants:
        applicants[x].shortlist()
    for x in houses:
        houses[x].contract()
    cycle += 1
    print(cycle)
    
    

KeyboardInterrupt: 

In [19]:
homes['status'].unique()

array(['Live'], dtype=object)

Run bidding cycle

In [65]:
for x in applicants:
    applicants[x].bidder(houses)

Run shortlisting cycle

In [66]:
for x in houses:
    houses[x].shortlister()
    

In [56]:
for x in houses:
    print(houses[x])

House ID:1 has 1 rooms and 28 bid(s). It is currently Live
House ID:2 has 5 rooms and 25 bid(s). It is currently Live
House ID:3 has 0 rooms and 43 bid(s). It is currently Live
House ID:4 has 5 rooms and 25 bid(s). It is currently Live
House ID:5 has 4 rooms and 11 bid(s). It is currently Live
House ID:6 has 2 rooms and 40 bid(s). It is currently Live
House ID:7 has 4 rooms and 7 bid(s). It is currently Live
House ID:8 has 4 rooms and 6 bid(s). It is currently Live
House ID:9 has 2 rooms and 40 bid(s). It is currently Live
House ID:10 has 1 rooms and 27 bid(s). It is currently Live
House ID:11 has 4 rooms and 7 bid(s). It is currently Live
House ID:12 has 4 rooms and 7 bid(s). It is currently Live
House ID:13 has 1 rooms and 24 bid(s). It is currently Live
House ID:14 has 4 rooms and 7 bid(s). It is currently Live
House ID:15 has 4 rooms and 12 bid(s). It is currently Live
House ID:16 has 4 rooms and 11 bid(s). It is currently Live
House ID:17 has 1 rooms and 17 bid(s). It is currently

In [67]:
houses[1].slist

,id,priority,bp_date,ap_date
0,189,1,2015-06-15,2022-03-21
1,56,1,2021-09-13,2022-02-15
2,71,1,2022-01-02,2021-12-09
3,115,1,2022-05-03,2022-02-11
4,148,2,2018-07-15,2022-03-08
5,187,3,2016-02-11,2022-05-21
6,167,3,2016-09-10,2022-05-28
7,25,3,2017-08-12,2021-09-23
8,97,3,2019-06-09,2022-02-12
9,121,3,2019-12-22,2022-02-01
